In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5959,2020-12-03T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1465,221,1686,38094,...,49,1294,26432,1650,0.0,0.0,67862,992813,674771.0,NaN
5960,2020-12-03T17:00:00,ITA,9,Toscana,43.769231,11.255889,1467,272,1739,33691,...,-2095,929,67618,2756,0.0,0.0,105804,1606027,971080.0,NaN
5961,2020-12-03T17:00:00,ITA,10,Umbria,43.106758,12.388247,344,54,398,6364,...,-321,286,17314,440,0.0,0.0,24516,428429,228438.0,NaN
5962,2020-12-03T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,113,11,124,994,...,-88,42,5177,324,0.0,0.0,6619,61669,34276.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5959,2020-12-03T17:00:00,19,Sicilia,1465,221,1686,38094,39780,49,1294,26432,1650,0.0,0.0,67862,992813,674771.0
5960,2020-12-03T17:00:00,9,Toscana,1467,272,1739,33691,35430,-2095,929,67618,2756,0.0,0.0,105804,1606027,971080.0
5961,2020-12-03T17:00:00,10,Umbria,344,54,398,6364,6762,-321,286,17314,440,0.0,0.0,24516,428429,228438.0
5962,2020-12-03T17:00:00,2,Valle d'Aosta,113,11,124,994,1118,-88,42,5177,324,0.0,0.0,6619,61669,34276.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-03', '2020-12-02')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-12-03T17:00:00,1,Piemonte,4459,366,4825,65181,70006,-881,2230,96473,6452,0.0,0.0,172931,1619813,982816.0
1,2020-12-03T17:00:00,2,Valle d'Aosta,113,11,124,994,1118,-88,42,5177,324,0.0,0.0,6619,61669,34276.0
2,2020-12-03T17:00:00,3,Lombardia,7025,836,7861,110470,118331,-465,3751,278058,22626,0.0,0.0,419015,4179837,2383936.0
3,2020-12-03T17:00:00,5,Veneto,2501,307,2808,67795,70603,1811,3581,79905,3982,0.0,0.0,154490,2841581,1077351.0
4,2020-12-03T17:00:00,6,Friuli Venezia Giulia,629,59,688,14195,14883,190,772,17146,941,0.0,0.0,32970,744835,290494.0
5,2020-12-03T17:00:00,7,Liguria,932,102,1034,10424,11458,-541,422,38887,2449,0.0,0.0,52794,614263,294667.0
6,2020-12-03T17:00:00,8,Emilia-Romagna,2668,245,2913,67324,70237,-1068,1766,51674,5959,0.0,0.0,127870,2174960,1097761.0
7,2020-12-03T17:00:00,9,Toscana,1467,272,1739,33691,35430,-2095,929,67618,2756,0.0,0.0,105804,1606027,971080.0
8,2020-12-03T17:00:00,10,Umbria,344,54,398,6364,6762,-321,286,17314,440,0.0,0.0,24516,428429,228438.0
9,2020-12-03T17:00:00,11,Marche,526,88,614,18335,18949,380,476,10793,1298,0.0,0.0,31040,420304,244290.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-12-02T17:00:00,1,Piemonte,4596,382,4978,65909,70887,-1382,1568,93454,6360,70823.0,99878.0,170701,1593650,971526.0
1,2020-12-02T17:00:00,2,Valle d'Aosta,122,13,135,1071,1206,-59,42,5049,322,5858.0,719.0,6577,61373,34160.0
2,2020-12-02T17:00:00,3,Lombardia,7222,855,8077,110719,118796,-2237,3425,274189,22279,305002.0,110262.0,415264,4143566,2372367.0
3,2020-12-02T17:00:00,5,Veneto,2519,308,2827,65965,68792,-12205,2782,78230,3887,37759.0,113150.0,150909,2819945,1070844.0
4,2020-12-02T17:00:00,6,Friuli Venezia Giulia,652,58,710,13983,14693,225,812,16616,889,27620.0,4578.0,32198,734865,286147.0
5,2020-12-02T17:00:00,7,Liguria,963,102,1065,10934,11999,-151,349,37950,2423,35020.0,17352.0,52372,609306,293385.0
6,2020-12-02T17:00:00,8,Emilia-Romagna,2692,247,2939,68366,71305,-537,1569,48931,5874,76963.0,49147.0,126110,2156981,1091519.0
7,2020-12-02T17:00:00,9,Toscana,1514,271,1785,35740,37525,-2676,776,64632,2718,79788.0,25087.0,104875,1591558,966235.0
8,2020-12-02T17:00:00,10,Umbria,342,62,404,6679,7083,-597,278,16722,425,6744.0,17486.0,24230,424921,227246.0
9,2020-12-02T17:00:00,11,Marche,565,85,650,17919,18569,365,421,10706,1289,30564.0,0.0,30564,417126,242628.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-12-03T17:00:00,1,Piemonte,4459,366,4825,65181,70006,-881,2230,96473,6452,0.0,0.0,172931,1619813,982816.0
1,2020-12-03T17:00:00,2,Valle d'Aosta,113,11,124,994,1118,-88,42,5177,324,0.0,0.0,6619,61669,34276.0
2,2020-12-03T17:00:00,3,Lombardia,7025,836,7861,110470,118331,-465,3751,278058,22626,0.0,0.0,419015,4179837,2383936.0
3,2020-12-03T17:00:00,4,Trentino-Alto Adige,832,80,912,12835,13747,-41,613,25989,1232,0.0,0.0,40968,702432,288589.0
4,2020-12-03T17:00:00,5,Veneto,2501,307,2808,67795,70603,1811,3581,79905,3982,0.0,0.0,154490,2841581,1077351.0
5,2020-12-03T17:00:00,6,Friuli Venezia Giulia,629,59,688,14195,14883,190,772,17146,941,0.0,0.0,32970,744835,290494.0
6,2020-12-03T17:00:00,7,Liguria,932,102,1034,10424,11458,-541,422,38887,2449,0.0,0.0,52794,614263,294667.0
7,2020-12-03T17:00:00,8,Emilia-Romagna,2668,245,2913,67324,70237,-1068,1766,51674,5959,0.0,0.0,127870,2174960,1097761.0
8,2020-12-03T17:00:00,9,Toscana,1467,272,1739,33691,35430,-2095,929,67618,2756,0.0,0.0,105804,1606027,971080.0
9,2020-12-03T17:00:00,10,Umbria,344,54,398,6364,6762,-321,286,17314,440,0.0,0.0,24516,428429,228438.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-12-02T17:00:00,1,Piemonte,4596,382,4978,65909,70887,-1382,1568,93454,6360,70823.0,99878.0,170701,1593650,971526.0
1,1,2020-12-02T17:00:00,2,Valle d'Aosta,122,13,135,1071,1206,-59,42,5049,322,5858.0,719.0,6577,61373,34160.0
2,2,2020-12-02T17:00:00,3,Lombardia,7222,855,8077,110719,118796,-2237,3425,274189,22279,305002.0,110262.0,415264,4143566,2372367.0
3,19,2020-12-02T17:00:00,4,Trentino-Alto Adige,828,80,908,12880,13788,39,595,25360,1207,32199.0,8156.0,40355,696139,286970.0
4,3,2020-12-02T17:00:00,5,Veneto,2519,308,2827,65965,68792,-12205,2782,78230,3887,37759.0,113150.0,150909,2819945,1070844.0
5,4,2020-12-02T17:00:00,6,Friuli Venezia Giulia,652,58,710,13983,14693,225,812,16616,889,27620.0,4578.0,32198,734865,286147.0
6,5,2020-12-02T17:00:00,7,Liguria,963,102,1065,10934,11999,-151,349,37950,2423,35020.0,17352.0,52372,609306,293385.0
7,6,2020-12-02T17:00:00,8,Emilia-Romagna,2692,247,2939,68366,71305,-537,1569,48931,5874,76963.0,49147.0,126110,2156981,1091519.0
8,7,2020-12-02T17:00:00,9,Toscana,1514,271,1785,35740,37525,-2676,776,64632,2718,79788.0,25087.0,104875,1591558,966235.0
9,8,2020-12-02T17:00:00,10,Umbria,342,62,404,6679,7083,-597,278,16722,425,6744.0,17486.0,24230,424921,227246.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-12-03T17:00:00,1,Piemonte,4459,366,4825,65181,70006,-881,2230,96473,6452,0.0,0.0,172931,1619813,982816.0,4341375
1,2020-12-03T17:00:00,2,Valle d'Aosta,113,11,124,994,1118,-88,42,5177,324,0.0,0.0,6619,61669,34276.0,125501
2,2020-12-03T17:00:00,3,Lombardia,7025,836,7861,110470,118331,-465,3751,278058,22626,0.0,0.0,419015,4179837,2383936.0,10103969
3,2020-12-03T17:00:00,4,Trentino-Alto Adige,832,80,912,12835,13747,-41,613,25989,1232,0.0,0.0,40968,702432,288589.0,1074819
4,2020-12-03T17:00:00,5,Veneto,2501,307,2808,67795,70603,1811,3581,79905,3982,0.0,0.0,154490,2841581,1077351.0,4907704
5,2020-12-03T17:00:00,6,Friuli Venezia Giulia,629,59,688,14195,14883,190,772,17146,941,0.0,0.0,32970,744835,290494.0,1211357
6,2020-12-03T17:00:00,7,Liguria,932,102,1034,10424,11458,-541,422,38887,2449,0.0,0.0,52794,614263,294667.0,1543127
7,2020-12-03T17:00:00,8,Emilia-Romagna,2668,245,2913,67324,70237,-1068,1766,51674,5959,0.0,0.0,127870,2174960,1097761.0,4467118
8,2020-12-03T17:00:00,9,Toscana,1467,272,1739,33691,35430,-2095,929,67618,2756,0.0,0.0,105804,1606027,971080.0,3722729
9,2020-12-03T17:00:00,10,Umbria,344,54,398,6364,6762,-321,286,17314,440,0.0,0.0,24516,428429,228438.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-12-03T17:00:00,1,Piemonte,4459,366,4825,65181,70006,-881,2230,96473,6452,0.0,0.0,172931,1619813,982816.0,4341375,575
1,2020-12-03T17:00:00,2,Valle d'Aosta,113,11,124,994,1118,-88,42,5177,324,0.0,0.0,6619,61669,34276.0,125501,20
2,2020-12-03T17:00:00,3,Lombardia,7025,836,7861,110470,118331,-465,3751,278058,22626,0.0,0.0,419015,4179837,2383936.0,10103969,1036
3,2020-12-03T17:00:00,4,Trentino-Alto Adige,832,80,912,12835,13747,-41,613,25989,1232,0.0,0.0,40968,702432,288589.0,1074819,106
4,2020-12-03T17:00:00,5,Veneto,2501,307,2808,67795,70603,1811,3581,79905,3982,0.0,0.0,154490,2841581,1077351.0,4907704,1016
5,2020-12-03T17:00:00,6,Friuli Venezia Giulia,629,59,688,14195,14883,190,772,17146,941,0.0,0.0,32970,744835,290494.0,1211357,180
6,2020-12-03T17:00:00,7,Liguria,932,102,1034,10424,11458,-541,422,38887,2449,0.0,0.0,52794,614263,294667.0,1543127,209
7,2020-12-03T17:00:00,8,Emilia-Romagna,2668,245,2913,67324,70237,-1068,1766,51674,5959,0.0,0.0,127870,2174960,1097761.0,4467118,634
8,2020-12-03T17:00:00,9,Toscana,1467,272,1739,33691,35430,-2095,929,67618,2756,0.0,0.0,105804,1606027,971080.0,3722729,523
9,2020-12-03T17:00:00,10,Umbria,344,54,398,6364,6762,-321,286,17314,440,0.0,0.0,24516,428429,228438.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-03T17:00:00,3,Lombardia,7025,836,7861,110470,118331,-465,3751,...,80.69,-249,326,36271,11569.0,347,3869,10.34,0.03712,4.15
1,2020-12-03T17:00:00,5,Veneto,2501,307,2808,67795,70603,1811,3581,...,30.22,1830,799,21636,6507.0,95,1675,16.55,0.07297,3.15
2,2020-12-03T17:00:00,15,Campania,2041,162,2203,100371,102574,79,2295,...,32.08,113,453,24709,10086876.0,54,2162,9.29,0.03967,2.78
3,2020-12-03T17:00:00,1,Piemonte,4459,366,4825,65181,70006,-881,2230,...,63.65,-728,662,26163,11290.0,92,3019,8.52,0.05137,3.98
4,2020-12-03T17:00:00,12,Lazio,3233,364,3597,89282,92879,975,1769,...,42.98,1010,-22,22793,8800.0,38,756,7.76,0.03016,2.13
5,2020-12-03T17:00:00,8,Emilia-Romagna,2668,245,2913,67324,70237,-1068,1766,...,38.64,-1042,197,17979,6242.0,85,2743,9.82,0.03953,2.86
6,2020-12-03T17:00:00,16,Puglia,1621,226,1847,39902,41749,989,1602,...,61.25,1031,-66,8753,2925.0,42,571,18.30,0.03997,1.48
7,2020-12-03T17:00:00,19,Sicilia,1465,221,1686,38094,39780,49,1294,...,37.59,77,-189,10581,6329.0,34,1211,12.23,0.02604,1.37
8,2020-12-03T17:00:00,9,Toscana,1467,272,1739,33691,35430,-2095,929,...,52.01,-2049,153,14469,4845.0,38,2986,6.42,0.02495,2.84
9,2020-12-03T17:00:00,6,Friuli Venezia Giulia,629,59,688,14195,14883,190,772,...,32.78,212,-40,9970,4347.0,52,530,7.74,0.06373,2.72


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,836,200,1036,80.69
1,Trentino-Alto Adige,80,26,106,75.47
2,Piemonte,366,209,575,63.65
3,Marche,88,55,143,61.54
4,Puglia,226,143,369,61.25
5,Valle d'Aosta,11,9,20,55.00
6,Toscana,272,251,523,52.01
7,Abruzzo,76,72,148,51.35
8,Liguria,102,107,209,48.80
9,Umbria,54,63,117,46.15


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,2020-12-01T17:00:00,22,P.A. Trento,406,44,450,2000,2450,2,156,12777,662,7922.0,7967.0,15889,377839,136084.0
564,2020-12-02T17:00:00,21,P.A. Bolzano,416,33,449,10880,11329,30,317,12317,542,24188.0,0.0,24188,314595,149819.0
565,2020-12-02T17:00:00,22,P.A. Trento,412,47,459,2000,2459,9,278,13043,665,8011.0,8156.0,16167,381544,137151.0
566,2020-12-03T17:00:00,21,P.A. Bolzano,428,35,463,10752,11215,-114,374,12789,558,0.0,0.0,24562,317314,150436.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2020-11-29T17:00:00,4,Trentino-Alto Adige,824,80,904,12752,13656,-35,702,...,19.0,7177.0,1569.0,-48.0,8.85,6.62,9.78,44.74,2.56,-0.26
280,2020-11-30T17:00:00,4,Trentino-Alto Adige,843,81,924,12846,13770,114,359,...,14.0,1739.0,617.0,94.0,8.77,6.71,20.64,58.18,1.25,0.83
281,2020-12-01T17:00:00,4,Trentino-Alto Adige,820,78,898,12851,13749,-21,250,...,11.0,3210.0,1054.0,5.0,8.69,6.53,7.79,23.72,-3.70,-0.15
282,2020-12-02T17:00:00,4,Trentino-Alto Adige,828,80,908,12880,13788,39,595,...,11.0,5716.0,1641.0,29.0,8.81,6.59,10.41,36.26,2.56,0.28


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
